In [1]:
import sys
import pathlib

sys.path.append(r"C:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition")

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

IMAGE_SIZE = 64
BATCH_SIZE = 64
DATASET_REPETITIONS = 5
LOAD_MODEL = False

NOISE_EMBEDDING_SIZE = 32
PLOT_DIFFUSION_STEPS = 20

# optimization
EMA = 0.999
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 50


In [18]:
import math

class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim, theta=10000):
        super().__init__()
        self.dim = dim
        self.theta = theta

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(self.theta) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        # emb = x[:, None] * emb[None, :]
        emb = x*emb
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

In [17]:
sin_emb = SinusoidalPosEmb(32,1000)
x = torch.tensor([0.34])
emb = sin_emb(x)
print(emb.shape)

torch.Size([32])


In [7]:
emb

tensor([[3.3349e-01, 2.1288e-01, 1.3494e-01, 8.5300e-02, 5.3860e-02, 3.3993e-02,
         2.1451e-02, 1.3535e-02, 8.5403e-03, 5.3886e-03, 3.4000e-03, 2.1453e-03,
         1.3536e-03, 8.5404e-04, 5.3886e-04, 3.4000e-04, 9.4275e-01, 9.7708e-01,
         9.9085e-01, 9.9636e-01, 9.9855e-01, 9.9942e-01, 9.9977e-01, 9.9991e-01,
         9.9996e-01, 9.9999e-01, 9.9999e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00]])

In [22]:
class SinusoidalEmbedding(nn.Module):
    def __init__(self, device, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.device = device
        thefreq = torch.log(torch.tensor(1000.0)/15)
        # frequencies = torch.linspace(torch.log(torch.tensor(1.0)),torch.log(torch.tensor(1000.0)),16)        
        frequencies = torch.arange(16) * -thefreq
        frequencies = frequencies.to(device)
        self.frequencies = frequencies
    
    def forward(self,x):
        angular_speeds = 2 * torch.pi * torch.exp(self.frequencies) * x
        # return single scalar as 32 dimensional vector
        return torch.cat((torch.sin(angular_speeds),torch.cos(angular_speeds)),dim=-1)


In [23]:
mysin_emb = SinusoidalEmbedding(x.device)
my_emb = mysin_emb(x)
print(my_emb.shape)

torch.Size([32])


In [24]:
my_emb

tensor([ 8.4433e-01,  3.2039e-02,  4.8066e-04,  7.2100e-06,  1.0815e-07,
         1.6222e-09,  2.4334e-11,  3.6500e-13,  5.4751e-15,  8.2126e-17,
         1.2319e-18,  1.8478e-20,  2.7717e-22,  4.1576e-24,  6.2364e-26,
         9.3547e-28, -5.3583e-01,  9.9949e-01,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00])